In [1]:
import pandas as pd
from regulations_rag.standard_regulation_index import load_parquet_data, load_index_data_from_files, save_parquet_data

In [12]:
path_to_definitions_as_parquet_file = "./inputs/definitions_gdpr.parquet"
path_to_additional_definitions_as_parquet_file = ""
path_to_index_as_parquet_file = "./inputs/index_gdpr.parquet"
path_to_additional_index_as_parquet_file = ""
path_to_workflow_as_parquet = ""

#decryption_key = ""
decryption_key = os.getenv('encryption_key_gdpr')

df_definitions, df_index, df_workflow = load_index_data_from_files(path_to_definitions_as_parquet_file, 
                                                                    path_to_additional_definitions_as_parquet_file, 
                                                                    path_to_index_as_parquet_file, 
                                                                    path_to_additional_index_as_parquet_file, 
                                                                    path_to_workflow_as_parquet, 
                                                                    decryption_key=decryption_key)

path_to_recitals_as_parquet_file = "./inputs/recitals_gdpr.parquet"
df_recitals = load_parquet_data(path_to_recitals_as_parquet_file)

In [13]:
# check the column names
from regulations_rag.standard_regulation_index import \
     required_columns_definition, \
     required_columns_index, \
     required_columns_workflow 

def check_columns(df, expected_column_names, description):
    contains_all_columns = True
    columns = df.columns.to_list()
    for name in expected_column_names:
        if name not in columns:
            print(f"{name} is a required column name for {description} but it does not appear in the regulation DataFrame")
            contains_all_columns = False
    if contains_all_columns:
        print(f"{description} contains all the mandatory columns")    

check_columns(df_definitions, required_columns_definition, "df_definitions")
check_columns(df_index, required_columns_index, "df_index")
#check_columns(df_workflow, required_columns_workflow, "df_workflow")
#check_columns(df_recitals, required_columns_recitals, "df_recitals")


df_definitions contains all the mandatory columns
df_index contains all the mandatory columns


Encrypt index

In [19]:
import os
gdrp_key = os.getenv("encryption_key_gdpr")
#save_parquet_data(df_index, path_to_index_as_parquet_file, gdrp_key)
#save_parquet_data(df_index, "./regs/original_index_unencrypted.parquet")
df_index.to_parquet("./regs/original_index_unencrypted.parquet")

In [ ]:
model = "text-embedding-3-large"
dimensions = 1024

increment = 10
for i in range(0, len(df_index), increment):
    chunk = df_index.iloc[i:i+increment].copy()
    chunk["embedding"] = chunk["text"].apply(lambda x: get_ada_embedding(openai_client, x, model, dimensions))
    df_index.loc[chunk.index, "embedding"] = chunk["embedding"]
    print(f"Completed {i+increment} lines")



In [ ]:
df_index.to_parquet('./inputs/index_gdpr.parquet', engine = 'pyarrow')

In [ ]:
df_recitals["embedding"] = df_recitals["text"].apply(lambda x: get_ada_embedding(openai_client, x, model, dimensions))
df_recitals.to_parquet('./inputs/recitals_gdpr.parquet', engine = 'pyarrow')

In [ ]:
df_definitions["embedding"] = df_definitions["text"].apply(lambda x: get_ada_embedding(openai_client, x, model, dimensions))
df_definitions.to_parquet('./inputs/definitions_gdpr.parquet', engine = 'pyarrow')
